# Lab05 - Tipos de Redes Neuronales  

En la sesión pasada creamos nuestra primera **Red Neuronal Profunda** usando capas densas (*Fully Connected*) ya que se trataba de una regresión lineal sencilla.  

Ahora, vamos a crear una **Red Neuronal Convolucional** en un problema muy común de Inteligencia Artificial: Clasificación de imágenes!  

In [ ]:
import keras
import cv2
import matplotlib.pyplot as plt
import numpy as np

from skimage import io
from PIL import Image
from io import BytesIO
from keras. preprocessing.image import ImageDataGenerator
%matplotlib inline

## Cargar el set de datos
Dependiendo del problema que queramos solucionar, va a ser necesario recolectar una gran cantidad de imágenes para entrenar el modelo.  Existen fuentes de datos abiertas para casos de uso generales como: personas, animales, flores, automóviles, etc.  
Por otro lado, cuándo nuestro problema es mucho más particular y no se encuentran datasets ya creados para nuestro caso, será necesario recolectar las imágenes manualmente.  La regla siempre será: *Entre más imágenes, mejor*.  
En cualquiera de los dos casos, la estructura recomendable para organizar las imágenes es la siguiente:
```
Directorio Raíz
└───train
│   └───clase1
│   │       imagen01.jpg
│   │       imagen02.jpg
│   │       ...
│   └───clase2
│   │       imagen01.jpg
│   │       imagen02.jpg
│   │       ...
│   └───...
│   
└───test
    └───clase1
    │       imagen01.jpg
    │       imagen02.jpg
    │       ...
    └───clase2
    │       imagen01.jpg
    │       imagen02.jpg
    │       ...
    └───...
```

Los directorios no *necesariemante* deben llamarse así, pero sí es recomendable tener un set de datos de entrenamiento separado del de pruebas.  Además, los **nombres** de las carpetas dentro de estos (clase1, clase2, ...), son los que indicarán las **clases/labels** de las imagenes que están adentro.

Vamos a trabajar con un dataset de Logos de algunas empresas conocidas, se puede descargar de [aquí](http://flovv.github.io/Logo_detection_deep_learning/), pero para facilidad ya se encuentra descargado y  organizado en la estructura necesaria dentro de nuestra carpeta */data*:
![Logos_sample](http://flovv.github.io/figures/post21/flickr27-sample.png)

El dataset consta de 675 imágenes de 27 marcas diferentes (En promedio 25 imágenes de cada clase para entrenamiento y pruebas).  En un problema real esto es un dataset pequeño, pero hoy nos servirá para aprender otros conceptos de Redes Neuronales.

## 5a. Mi Primera Red Neuronal Convolucional CNN

In [ ]:
# Configuración de variables
img_width = 32
img_height = 32
train_samples = 498
validation_samples = 177
classes = ['Adidas', 'Apple', 'BMW', 'Citroen', 'Cocacola', 'DHL', 'Fedex', 'Ferrari', 'Ford', 'Google', 
           'Heineken', 'HP', 'Intel', 'McDonalds', 'Mini', 'Nbc', 'Nike', 'Pepsi', 'Porsche', 'Puma', 
           'RedBull', 'Sprite', 'Starbucks', 'Texaco', 'Unicef', 'Vodafone', 'Yahoo']

train_directory = "data/LogoData/train"
test_directory = "data/LogoData/test"

In [ ]:
# Configuración de Hiperparámetros:
batch_size = 8
learning_rate = 0.0001
epochs = 50

### Data Augmentation
Como ya se ha mencionado varias veces, entre más datos tengamos, mejor.  En el caso de Computer Vision: entre más imágenes, mejor.
Cuando no tenemos suficientes datos, la técnica de Data Augmentation permite, por medio de un algoritmo, generar imágenes artificialmente a partir de las pocas imágenes que tenemos.  Esto se logra haciendo cambios pequeños a los datos: Girar un poco la imagen, cambiando su luminocidad, resolución, moviéndola un poco, etc.  Para nosotros es un cambio pequeño, pero para el modelo es una imagen completamente diferente!
![Data Augmenation](https://miro.medium.com/max/605/0*Utma-dS47hSoQ6Zt)  
A nivel de código, estas son TODAS las opciones que se pueden configurar para el generador de datos, tenga en cuenta que no siempre todas aplican, por ejemplo, la imagen del león mirando a la derecha o mirando a la izquierda, sigue siendo un león, pero, nuestros logos?

In [ ]:
# Estas son TODAS las opciones!
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None
)

In [ ]:
datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [ ]:
train_generator = datagen.flow_from_directory(
    directory=train_directory,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = True,
    seed = 123
)

validation_generator = datagen.flow_from_directory(
    directory=test_directory,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = True,
    seed = 123
)

Hemos creado dos generadores de imagenes porque cada dataset partirá de un directorio diferente, y puede ser con transformaciones diferentes.  
Ahora sí, podemos crear el modelo:

In [ ]:
model = keras.Sequential()

model.add(keras.layers.convolutional.Conv2D(filters=16, kernel_size = (3,3), 
                                            input_shape=(img_width, img_height,3)))
model.add(keras.layers.Activation(activation='relu'))
model.add(keras.layers.convolutional.MaxPooling2D(pool_size = (2,2)))

model.add(keras.layers.convolutional.Conv2D(filters=32, kernel_size = (3,3)))
model.add(keras.layers.Activation(activation='relu'))
model.add(keras.layers.convolutional.MaxPooling2D(pool_size = (2,2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64))
model.add(keras.layers.Activation(activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(27))
model.add(keras.layers.Activation(activation='softmax'))

In [ ]:
model.summary()

En el resumen que nos entrega el modelo podemos ver:
- Estructura de la red, capa por capa
- Como van disminuyendo las dimensiones de cada capa, empezando por las dimensiones de la imágen (30,30) hasta la última capa = Número de clases (27)
- Cantidad de parámetros que se van a calcular/entrenar: 80.635

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.rmsprop(learning_rate=learning_rate),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, 
                    steps_per_epoch = int(train_samples/batch_size), 
                    epochs = epochs, 
                    validation_data = validation_generator,
                    validation_steps = int(validation_samples/batch_size))

In [ ]:
plt.xlabel('# Epoca')
plt.ylabel("Función de Pérdida (loss)")
plt.plot(history.history['loss'])

En la gráfica de **loss** podemos ver cómo va disminuyendo nuestro *categorical_crossentropy* en cada epoca.  
También podemos ver los valores finales del loss y el accuracy:

In [ ]:
model.evaluate_generator(validation_generator, validation_samples)

Una vez aprobemos nuestro modelo, podemos usarlo para predecir sobre imágenes nuevas:

In [ ]:
img_path  = "https://i.pinimg.com/originals/07/0d/3e/070d3e118fa9bcf14bf6a1004308c91b.jpg"
response = requests.get(img_path)
img = Image.open(BytesIO(response.content))
plt.imshow(img)

In [ ]:
img = io.imread(img_path)
img = cv2.resize(img, (img_width, img_height)).astype(np.float32)
img = np.expand_dims(img, axis=0)
prediction = model.predict(img)
prediction

El resultado de la predicción es la salida de la última neurona, es decir 27 valores que siginifican la probabilidad de que la imagen dada pertenesca a cada una de las 27 clases objetivo.  

In [ ]:
len(prediction[0])

Para saber, entonces, la clase que indica la predicción basta con elegir la probabilidad más alta del vector entregado, pues corresponden, por índice a las clases objetivo:

In [ ]:
classes[prediction.argmax()]

**Taller:**  
A pesar de que para este ejemplo, el modelo hizo una predicción correcta, el loss aún está un poco alto, y el accuracy esta muy bajo.  Además, el valor de *accuracy* para el set de entrenamiento fue más alto que el de pruebas, quiere decir que el modelo está aprendiendo de memoria casos particulres, en lugar de patrones generales (Overfit).  Para ello:
- Juegue un poco con los hiperparámetros, capas y funciones de activación para mejorar estos valores.  Para ello peude usar la documentación de [keras](https://keras.io/api/)
- Busque otra imagen (*.jpg*) de alguno de los logos en nuestro modelo y haga la predicción.  Funciona? Estamos safisfechos con este modelo? Podria mejorar?


## 5b. Transfer Learning  
Para no tener que diseñar la red desde ceros, pensando qué capas, neuronas y funciones de activación usar, usamos Transfer Learning, en el que tomamos la estructura de una red ya contruida y especializada en una tarea específica (Ej: Reconocimiento de imagen).  
Los siguientes pasos iniciales siguen siendo necesarios: 
- Definición de hiperprámetros
- Generador de imagenes nuevas (Data Augemtnation)
- Configuración de generadores para entrenamiento y pruebas  

Pero en lugar de crear un modelo desde ceros, vamos a tomar un modelo ya existente, por ejemplo VGG16 y vamos a indicar que sus capas NO SON reentrenables:

In [ ]:
base_model = keras.applications.VGG16(weights='imagenet', include_top=False)

for layer in base_model.layers :
    layer.trainable = False

base_model.summary()

Vemos la estructura tan compleja que tiene esta red ya preentrenada, más de 14.5 millones de parámetros!  En este caso, también hemos indicado que este modelo no sea reentrenable.  
También hemos indicado al modelo no incluir la "cabeza" de la red, pues las últimas capas las vamos a reescribir nosotros mismos:
> Ya hemos visto una forma de definir la red: deinifiendo el modelo Sequencil vacío e ir añadiendo capas (*model.add*).  
> Otra forma es definiendo cada capa, y agregándola como entrada de la capa siguiente:

In [ ]:
input = keras.Input(shape=(img_width, img_height, 3),name = 'image_input')

model_head = base_model(input)
model_head = keras.layers.GlobalAveragePooling2D()(model_head)
model_head = keras.layers.Dense(64)(model_head)
model_head = keras.layers.Activation(activation='relu')(model_head)
model_head = keras.layers.Dropout(0.4)(model_head)
model_head = keras.layers.Dense(27)(model_head)
model_head = keras.layers.Activation(activation='softmax')(model_head)

my_model = keras.Model(input=input, output=model_head)

In [ ]:
my_model.summary()

Vemos ahora que, la estructura de nuestra nueva red es mucho más compleja, incluye los 14.5 millones de parámetros del modelo VGG16, adicionalmente hemos configurado la capa de entrada para que reciba las imágenes que tenemos preparadas, pero lo más importante, hemos configurado algunas capas adicionales como la **nueva cabeza** de la red.  
Al final del resumen vemos que, los 14.5 millones de parámetros del modelo base (VGG16) no son entrenables, pero los 34mil de nuestra nueva cabeza sí, y estos son los valores que va a buscar el modelo.  

Hemos creado en pocas líneas un modelo más complejo, pero más sencillo de entrenar.  Desde aquí la compilación y entrenamiento del modelo es igual a como ya lo hemos trabajado:

In [ ]:
my_model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.rmsprop(learning_rate=learning_rate),
              metrics=['accuracy'])

Como paso adicional de este laboratorio vamos a trabajar con los Callbacks que vimos la sesión pasada.  Son funciones que se ejecutan al final de cada época y que pueden ayudar a mejorar el rendimiento del modelo:

In [ ]:
mis_callbacks = [
    # Modificar el LR si el modelo no mejora:
    keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor=.1, patience=5),
    # Configurar que el modelo pare si no mejora:
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, mode='auto'),
    # Ir guardando el mejor modelo
    keras.callbacks.ModelCheckpoint("models/logos_checkpoints.h5", monitor='val_loss', save_best_only=True)
]

In [ ]:
history = my_model.fit(train_generator, 
                    steps_per_epoch = int(train_samples/batch_size), 
                    epochs = epochs, 
                    validation_data = validation_generator,
                    validation_steps = int(validation_samples/batch_size),
                    callbacks = mis_callbacks)

In [ ]:
plt.xlabel('# Epoca')
plt.ylabel("Función de Pérdida (loss)")
plt.plot(history.history['loss'])

In [ ]:
plt.xlabel('# Epoca')
plt.ylabel("Métrica (Accuracy)")
plt.plot(history.history['accuracy'])

In [ ]:
my_model.evaluate_generator(validation_generator, validation_samples)

En comparación con nuestro modelo inicial (*Loss = 2.46, Accuracy=0.26*), este modelo da mejores resultados!  
Vamos ahora a probarlo con imágenes nuevas:

In [ ]:
img_path  = "https://i.pinimg.com/originals/02/ac/cd/02accd95989df4cde2f57adcd508dbcd.jpg"
response = requests.get(img_path)
img = Image.open(BytesIO(response.content))
plt.imshow(img)

In [ ]:
img = io.imread(img_path)
img = cv2.resize(img, (img_width, img_height)).astype(np.float32)
img = np.expand_dims(img, axis=0)
prediction = my_model.predict(img)
prediction

In [ ]:
len(prediction[0])

In [ ]:
classes[prediction.argmax()]

**Taller**  
**Ups!** Al modelo le falta un poco de afinamiento, intente:
- Cambiar el modelo base VGG16 por otros modelos como *VGG19, ResNet50, InceptionV3, MobileNet, Xception*.  Para ello peude usar la documentación de [keras](https://keras.io/api/)
- Juegue un poco con los hiperparámetros, capas y funciones de activación en la **cabeza de la red** para mejorar los valores de loss, accuracy y las predicciones.
- Busque otra imagen (*.jpg*) de alguno de los logos en nuestro modelo y haga la predicción.  Funciona? Estamos safisfechos con este modelo? Podria mejorar?